In [12]:
## Code base used as reference: https://python.langchain.com/docs/integrations/providers/cohere/
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()
model="command-xlarge"

## Cohere Embedding Model Options:  
Cohere has two main “families” of generative models—Command and Base—each with different sizes (light, medium, xlarge).
1. Command Family
    * command-light
    * command-medium
    * command-xlarge
    * (Sometimes referred to simply as command when using the default version)
2. Base Family
    * base-light
    * base-medium
    * base-xlarge
    * (Sometimes referred to simply as base when using the default version)

In [13]:
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage
chat = ChatCohere(model=model)
messages = [HumanMessage(content="knock knock")]
print(chat.invoke(messages).content)

Who's there?


## LLM

In [14]:
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage
chat = ChatCohere(model=model)
messages = [HumanMessage(model)]
print(chat.invoke(messages).content)

Command is the name of Cohere's flagship large language model, but there is no variant of Command named 'X-Large'.


In [15]:
from langchain_cohere.llms import Cohere

llm = Cohere()
print(llm.invoke("Come up with a pet name"))

 Of course! Here are a few suggestions for pet names: 

- Bubbles 
- Cutie Pie
- Sweetie
- Honey Bunch
- Baby Girl/Boy

These are just a few suggestions, you may want to consider something that relates to a positive memory or experience you have shared together. 


## Tool calling

In [16]:
from langchain_cohere import ChatCohere
from langchain_core.messages import (
    HumanMessage,
    ToolMessage,
)
from langchain_core.tools import tool

@tool
def magic_function(number: int) -> int:
    """Applies a magic operation to an integer

    Args:
        number: Number to have magic operation performed on
    """
    return number + 10

def invoke_tools(tool_calls, messages):
    for tool_call in tool_calls:
        selected_tool = {"magic_function":magic_function}[
            tool_call["name"].lower()
        ]
        tool_output = selected_tool.invoke(tool_call["args"])
        messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
    return messages

tools = [magic_function]

llm = ChatCohere()
llm_with_tools = llm.bind_tools(tools=tools)
messages = [
    HumanMessage(
        content="What is the value of magic_function(2)?"
    )
]

res = llm_with_tools.invoke(messages)
while res.tool_calls:
    messages.append(res)
    messages = invoke_tools(res.tool_calls, messages)
    res = llm_with_tools.invoke(messages)

print(res.content)

The value of magic_function(2) is **12**.


## ReACT

In [17]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_cohere import ChatCohere, create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor

llm = ChatCohere()

internet_search = TavilySearchResults(max_results=4)
internet_search.name = "internet_search"
internet_search.description = "Route a user query to the internet"

prompt = ChatPromptTemplate.from_template("{input}")

agent = create_cohere_react_agent(
    llm,
    [internet_search],
    prompt
)

agent_executor = AgentExecutor(agent=agent, tools=[internet_search], verbose=True)

agent_executor.invoke({
    "input": "In what year was the company that was founded as Sound of Music added to the S&P 500?",
})



> Entering new AgentExecutor chain...

I will search for 'In what year was the company that was founded as Sound of Music added to the S&P 500?'
{'tool_name': 'internet_search', 'parameters': {'query': 'In what year was the company that was founded as Sound of Music added to the S&P 500?'}}
[{'url': 'https://www.facebook.com/Commish.Joe.Atkins/posts/57-years-ago-today-west-st-paul-resident-richard-schulze-founded-the-sound-of-mu/719189290019721/', 'content': '57 years ago today. West St. Paul resident Richard Schulze founded the Sound of Music, opening his first store in town on August 22, 1966. The company expanded to nine stores, and after a tornado'}, {'url': 'https://historydraft.com/story/best-buy/timeline/841', 'content': "Best Buy announced the shutdown of the Future Shop chain in Canada\nOn March 28, 2015, Best Buy announced the shutdown of the Future Shop chain in Canada; 65 of its 131 former locations were converted into Best Buy locations, while the rest (primarily those i

{'input': 'In what year was the company that was founded as Sound of Music added to the S&P 500?',
 'output': 'Best Buy, the company that was founded as Sound of Music, was added to the S&P 500 on June 29, 1999.',
 'citations': [CohereCitation(start=0, end=8, text='Best Buy', documents=[{'url': 'https://historydraft.com/story/best-buy/timeline/841', 'content': "Best Buy announced the shutdown of the Future Shop chain in Canada\nOn March 28, 2015, Best Buy announced the shutdown of the Future Shop chain in Canada; 65 of its 131 former locations were converted into Best Buy locations, while the rest (primarily those in close proximity to an existing Best Buy) were closed permanently.\n Best Buy Company, Inc\nIn 1983, with seven stores and $10 million in annual sales, Sound of Music was renamed Best Buy Company, Inc.\nBest Buy debuted on the New York Stock Exchange\nBest Buy was taken public in 1985, and two years later it debuted on the New York Stock Exchange.\n The company closed all o

## Cohere Embedding Model Options:  
- embed-english-v2.0 : General-purpose embeddings for English text.

- embed-english-light-v2.0
A lighter-weight model for English, optimized for speed and smaller resource usage.

- embed-multilingual-v2.0
Designed for multilingual text (supporting multiple languages), enabling cross-lingual semantic tasks.

- embed-multilingual-light-v2.0
A lighter-weight multilingual model, similar to embed-english-light-v2.0 but supporting multiple languages.

In [18]:
from langchain_cohere import ChatCohere
from langchain.retrievers import CohereRagRetriever
from langchain_core.documents import Document

rag = CohereRagRetriever(llm=ChatCohere(model=model), model="embed-english-v2.0")
print(rag.invoke("What is cohere ai?")[0].page_content)

/var/folders/1t/7cjrsy2949n9nwlmm_7d_c6c0000gn/T/ipykernel_82926/3327221834.py:5: LangChainDeprecationWarning: The class `CohereRagRetriever` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRagRetriever``.
  rag = CohereRagRetriever(llm=ChatCohere(model=model), model="embed-english-v2.0")
/Users/denizaskin/Knowledge-Graph/.venv/lib/python3.11/site-packages/langchain_core/language_models/chat_models.py:851: LangChainDeprecationWarning: The 'connectors' parameter is deprecated as of version 0.3.3.
Please use the 'tools' parameter instead.
  result = self._generate(


Cohere Inc. is a Canadian multinational technology company focused on artificial intelligence for the enterprise, specializing in large language models. Cohere was founded in 2019 by Aidan Gomez, Ivan Zhang, and Nick Frosst, and is headquartered in Toronto and San Francisco, with offices in Palo Alto, London, and New York City.

In 2017, a team of researchers at Google Brain introduced the transformer machine learning architecture in "Attention Is All You Need," which demonstrated state-of-the-art performance on a variety of natural language processing tasks. In 2019, Aidan Gomez, one of its co-authors, along with Nick Frosst, another researcher at Google Brain, founded Cohere with Ivan Zhang, with whom Gomez had done research at FOR.ai. All of the co-founders attended University of Toronto.


In [19]:
# pip install cohere
import cohere
co = cohere.ClientV2(api_key="COHERE_API_KEY") # Get your free API key: https://dashboard.cohere.com/api-keys

# Define the documents
faqs_short = [
    {"text": "Reimbursing Travel Expenses: Easily manage your travel expenses by submitting them through our finance tool. Approvals are prompt and straightforward."},
    {"text": "Working from Abroad: Working remotely from another country is possible. Simply coordinate with your manager and ensure your availability during core hours."},
    {"text": "Health and Wellness Benefits: We care about your well-being and offer gym memberships, on-site yoga classes, and comprehensive health insurance."},
    {"text": "Performance Reviews Frequency: We conduct informal check-ins every quarter and formal performance reviews twice a year."}
]

# Add the user query
query = "Are there fitness-related perks?"
# Rerank the documents
results = co.rerank(query=query,
                    documents=faqs_short,
                    top_n=2,
                    model='rerank-english-v3.0')
print(results)

# Display the reranking results
def return_results(results, documents):    
    for idx, result in enumerate(results.results):
        print(f"Rank: {idx+1}") 
        print(f"Score: {result.relevance_score}")
        print(f"Document: {documents[result.index]}\n")
    
return_results(results, faqs_short)

UnauthorizedError: status_code: 401, body: {'message': 'invalid api token'}